In [28]:
import cloudscraper
from bs4 import BeautifulSoup

def teste_conexao():
    url = "https://br.investing.com/equities/petrobras-pn-news/1"
    print(f"🔌 Tentando conectar em: {url}")
    
    # Cria o scraper (simula um navegador Chrome)
    scraper = cloudscraper.create_scraper()
    
    try:
        resp = scraper.get(url)
        print(f"📡 Status Code: {resp.status_code}")
        
        if resp.status_code != 200:
            print("❌ Bloqueio detectado pelo código de status.")
            return

        soup = BeautifulSoup(resp.text, 'html.parser')
        
        # Teste 1: Verifica Título da Página
        print(f"📄 Título da Página: {soup.title.string if soup.title else 'Sem Título'}")
        
        # Teste 2: Verifica se é página de bloqueio
        texto_pagina = resp.text.lower()
        if "just a moment" in texto_pagina or "access denied" in texto_pagina or "cloudflare" in texto_pagina:
            print("⛔ ALERTA: O Cloudflare capturou a requisição (Bloqueio Anti-Robô).")
            return
            
        # Teste 3: Testa Seletores de Notícia
        print("\n🕵️ Testando Seletores CSS:")
        
        selectors = [
            'article.articleItem',       # Antigo padrão
            'div.textDiv',               # Alternativo antigo
            'ul[data-test="news-list"]', # Novo layout possível
            'div.news-analysis-v2_content__z0_23', # Layout 2024
            'li.border-b'                # Layout genérico lista
        ]
        
        encontrou_algo = False
        for sel in selectors:
            items = soup.select(sel)
            print(f"   -> Seletor '{sel}': Encontrou {len(items)} elementos.")
            if len(items) > 0:
                encontrou_algo = True
                print(f"      Exemplo: {items[0].text[:50].strip()}...")
        
        if not encontrou_algo:
            print("\n❌ NENHUM SELETOR FUNCIONOU. O site mudou o HTML ou não carregou as notícias.")
            # Salva o HTML para você ver o que veio
            with open("debug_pagina_investing.html", "w", encoding="utf-8") as f:
                f.write(resp.text)
            print("💾 Salvei o HTML recebido em 'debug_pagina_investing.html'. Abra no navegador para ver o que o robô está vendo.")
        else:
            print("\n✅ SUCESSO! Encontramos notícias. O problema era apenas o seletor.")

    except Exception as e:
        print(f"🔥 Erro fatal na conexão: {e}")

if __name__ == "__main__":
    teste_conexao()

🔌 Tentando conectar em: https://br.investing.com/equities/petrobras-pn-news/1
📡 Status Code: 200
📄 Título da Página: Petrobras PN Notícias (PETR4) - Investing.com
⛔ ALERTA: O Cloudflare capturou a requisição (Bloqueio Anti-Robô).


In [29]:
import cloudscraper
from bs4 import BeautifulSoup

def investigar_layout():
    url = "https://br.investing.com/equities/petrobras-pn-news/1"
    print(f"🕵️ Investigando layout de: {url}")
    
    scraper = cloudscraper.create_scraper()
    
    try:
        resp = scraper.get(url)
        if resp.status_code != 200:
            print(f"❌ Erro de status: {resp.status_code}")
            return

        soup = BeautifulSoup(resp.text, 'html.parser')
        print(f"📄 Título confirmado: {soup.title.string.strip()}")
        
        # ESTRATÉGIA: Encontrar qualquer link que pareça uma notícia
        # Links de notícia no investing geralmente começam com /news/
        print("\n--- 🔍 TENTANDO ENCONTRAR NOTÍCIAS PELO LINK ---")
        
        links_candidatos = []
        all_links = soup.find_all('a', href=True)
        
        for link in all_links:
            href = link['href']
            text = link.text.strip()
            
            # Filtros para achar notícias reais e ignorar menus
            if '/news/' in href and len(text) > 20 and 'comentários' not in text.lower():
                links_candidatos.append(link)

        print(f"Encontrados {len(links_candidatos)} links potenciais de notícia.")
        
        if len(links_candidatos) > 0:
            print("\n✅ ESTRUTURA DETECTADA (Primeiro item):")
            exemplo = links_candidatos[0]
            print(f"   Manchete: {exemplo.text}")
            print(f"   Link: {exemplo['href']}")
            
            # Descobrir a classe do PAI (o container da notícia)
            pai = exemplo.parent
            print(f"   Tag Pai: <{pai.name} class='{pai.get('class') if pai.get('class') else 'SEM_CLASSE'}'>")
            
            avo = pai.parent
            if avo:
                print(f"   Tag Avô: <{avo.name} class='{avo.get('class') if avo.get('class') else 'SEM_CLASSE'}'>")
                
            bisavo = avo.parent
            if bisavo:
                print(f"   Tag Bisavô: <{bisavo.name} class='{bisavo.get('class') if bisavo.get('class') else 'SEM_CLASSE'}'>")
                
            print("\n💡 DICA PARA MIM (IA): Use as classes acima para consertar o scraper.")
        else:
            print("❌ Ainda não achei nada. O HTML pode estar sendo renderizado via Javascript.")
            
    except Exception as e:
        print(f"Erro: {e}")

if __name__ == "__main__":
    investigar_layout()

🕵️ Investigando layout de: https://br.investing.com/equities/petrobras-pn-news/1
📄 Título confirmado: Petrobras PN Notícias (PETR4) - Investing.com

--- 🔍 TENTANDO ENCONTRAR NOTÍCIAS PELO LINK ---
Encontrados 11 links potenciais de notícia.

✅ ESTRUTURA DETECTADA (Primeiro item):
   Manchete: Recomendações de analistas
   Link: https://br.investing.com/news/analyst-ratings
   Tag Pai: <li class='['flex', 'items-center', 'navbar_multi_list_list__BidbT']'>
   Tag Avô: <ul class='SEM_CLASSE'>
   Tag Bisavô: <div class='SEM_CLASSE'>

💡 DICA PARA MIM (IA): Use as classes acima para consertar o scraper.


In [30]:
import pandas as pd
import glob
import os
import re
from pathlib import Path
from datetime import datetime
from tqdm import tqdm
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [31]:

# --- CONFIGURAÇÃO ---
INPUT_DIR = "../data/news/investing"
OUTPUT_DIR = "../data/news/processed/investing"
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

analyzer = SentimentIntensityAnalyzer()

In [32]:
def limpar_data_json_sujo(raw_date):
    """
    Limpa datas que vieram sujas do JSON Mode.
    Entrada: '2025-12-20 01:31:03","writerId":0...'
    Saída: datetime(2025, 12, 20)
    """
    if not isinstance(raw_date, str): return pd.NaT
    
    texto = raw_date.strip()
    
    try:
        # Padrão ISO (YYYY-MM-DD) - Comum no JSON
        # Procura 4 digitos - 2 digitos - 2 digitos logo no começo ou meio
        match_iso = re.search(r'(\d{4})-(\d{2})-(\d{2})', texto)
        if match_iso:
            return datetime.strptime(match_iso.group(0), '%Y-%m-%d')

        # Padrão Brasileiro com Ponto (DD.MM.YYYY)
        match_pontos = re.search(r'(\d{2})\.(\d{2})\.(\d{4})', texto)
        if match_pontos:
            return datetime.strptime(match_pontos.group(0), '%d.%m.%Y')
            
        # Padrão Brasileiro com Barra (DD/MM/YYYY)
        match_barras = re.search(r'(\d{2})/(\d{2})/(\d{4})', texto)
        if match_barras:
            return datetime.strptime(match_barras.group(0), '%d/%m/%Y')
            
        # Relativos ("5 horas atrás") -> Assume Hoje
        texto_lower = texto.lower()
        if 'atrás' in texto_lower or 'ago' in texto_lower or 'min' in texto_lower:
            return datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
            
    except:
        pass
        
    return pd.NaT

In [33]:
def calcular_vader(texto):
    if not isinstance(texto, str): return 0.0
    return analyzer.polarity_scores(texto)['compound']

In [34]:
def processar_investing_v4():
    print("--- 🧹 INICIANDO SANEAMENTO INVESTING (V4 - FIX JSON SUJO) ---")
    arquivos = glob.glob(os.path.join(INPUT_DIR, "*.parquet"))
    
    count_sucesso = 0
    count_vazios = 0
    
    for arquivo in tqdm(arquivos, desc="Limpando"):
        nome_arq = os.path.basename(arquivo)
        try:
            df = pd.read_parquet(arquivo)
            
            if df.empty or 'raw_date' not in df.columns:
                count_vazios += 1
                continue

            # 1. Limpeza Cirúrgica da Data
            df['date'] = df['raw_date'].apply(limpar_data_json_sujo)
            
            # Remove quem ficou sem data
            df_limpo = df.dropna(subset=['date']).copy()
            
            if df_limpo.empty:
                # print(f"⚠️ {nome_arq}: Todas as datas falharam. Ex: {df['raw_date'].iloc[0][:50]}...")
                count_vazios += 1
                continue

            # 2. Sentimento
            df_limpo['sentiment'] = df_limpo['title'].apply(calcular_vader)

            # 3. Seleção de Colunas
            cols_finais = ['ticker', 'date', 'title', 'sentiment', 'link']
            # Cria colunas faltantes se precisar
            for col in cols_finais:
                if col not in df_limpo.columns: df_limpo[col] = None
            
            df_limpo = df_limpo[cols_finais]

            # 4. Salva (Agora vai funcionar!)
            df_limpo.to_parquet(os.path.join(OUTPUT_DIR, nome_arq), index=False)
            count_sucesso += 1
            
        except Exception as e:
            print(f"❌ Erro em {nome_arq}: {e}")

    print("-" * 40)
    print(f"📊 RESUMO FINAL:")
    print(f"✅ Arquivos salvos em 'processed': {count_sucesso}")
    print(f"⚠️ Arquivos ignorados (vazios): {count_vazios}")

if __name__ == "__main__":
    processar_investing_v4()

--- 🧹 INICIANDO SANEAMENTO INVESTING (V4 - FIX JSON SUJO) ---


Limpando: 100%|██████████| 23/23 [00:01<00:00, 18.85it/s]

----------------------------------------
📊 RESUMO FINAL:
✅ Arquivos salvos em 'processed': 23
⚠️ Arquivos ignorados (vazios): 0


In [35]:
import pandas as pd
import glob
import os

PATH_RAW = "../data/news/investing"

def diagnosticar():
    arquivos = glob.glob(os.path.join(PATH_RAW, "*.parquet"))
    
    if not arquivos:
        print("❌ Nenhum arquivo encontrado na pasta raw.")
        return

    print(f"📂 Total de arquivos brutos: {len(arquivos)}")
    
    # Pega os 3 primeiros arquivos não vazios para garantir
    count = 0
    for arq in arquivos:
        if count >= 3: break
        
        try:
            df = pd.read_parquet(arq)
            if df.empty:
                print(f"⚠️ {os.path.basename(arq)} está VAZIO.")
                continue
                
            print(f"\n--- 📄 {os.path.basename(arq)} ---")
            print(f"Linhas: {len(df)}")
            print("Colunas:", list(df.columns))
            
            if 'raw_date' in df.columns:
                print("🔎 Amostra de Datas Brutas:")
                print(df['raw_date'].head(10).tolist())
            else:
                print("❌ ERRO CRÍTICO: Coluna 'raw_date' não existe!")
            
            count += 1
        except Exception as e:
            print(f"Erro ao ler {arq}: {e}")

if __name__ == "__main__":
    diagnosticar()

📂 Total de arquivos brutos: 23

--- 📄 news_investing_ABEV3.parquet ---
Linhas: 1
Colunas: ['title', 'link', 'raw_date', 'source', 'ticker', 'slug']
🔎 Amostra de Datas Brutas:
['2025-12-20 01:31:03","writerId":0,"front_writer_name":"","front_member_name":"","front_writer_link":"","writerName":"","writerLink":"","writerImage":"","userFirstName":"","userLastName":"","writerArticleCount":0,"company_name":"","company_name_prepared":"","company_href":"","company_url":"","user_type":"","authorLink":"","editorId":0,"editorFirstName":"","editorLastName":"","editorLink":"","image_copyright":"\\u0026copy; REUTERS","related_image_big":"TheNewYorkStockExchange_800x533_L_1646233225.jpg","images_serialized":"a:1:{i:0;a:2:{s:3:\\"src\\";s:48:\\"TheNewYorkStockExchange_800x533_L_1646233225.jpg\\";s:5:\\"width\\";i:200;}}","imageHref":"https://i-invdn-com.investing.com/news/TheNewYorkStockExchange_800x533_L_1646233225.jpg']

--- 📄 news_investing_AZZA3.parquet ---
Linhas: 1
Colunas: ['title', 'link', 'ra